In [1]:
import os
import sys
import numpy as np
import pandas as pd

import math

sys.path.append('../')
from utils import *
from save_data import *
pd.set_option('display.max_rows', None)
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

2024-07-12 15:56:45.307471: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 15:56:45.310056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2024-07-12 15:56:45.310066: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
df = pd.read_csv('../expt_phys_data/full_dataset_with_scaffold_split.csv').fillna('')
df = df[(df.smiles != 'C') & (df.smiles != 'S') & (df.smiles != 'N')]
df.describe()

,expt,uncertainty,tip3p,cha,zap9,mbondi,asc,igb5,null,paper_split,hilo_split,scaffold_split,uncertain_expt_value
count,639.000000,639.000000,639.000000,639.000000,639.000000,639.000000,639.000000,639.000000,639.0,639.000000,639.000000,639.000000,639.000000
mean,-3.816182,0.567825,-3.498354,-3.979271,-3.770091,-4.461795,-3.960598,-5.199393,0.0,0.375587,0.375587,0.375587,0.057903
std,3.847976,0.306195,4.188449,4.110157,4.125217,4.595776,4.963090,4.753827,0.0,0.696902,0.696902,0.696902,0.233743
min,-25.470000,0.030000,-21.762000,-20.490600,-22.504700,-31.910700,-25.772000,-25.222000,0.0,0.000000,0.000000,0.000000,0.000000
25%,-5.730000,0.600000,-5.439000,-5.662690,-5.893780,-6.841250,-6.233500,-7.410500,0.0,0.000000,0.000000,0.000000,0.000000
50%,-3.540000,0.600000,-3.054000,-3.368530,-3.011160,-3.821480,-3.180000,-4.437000,0.0,0.000000,0.000000,0.000000,0.000000
75%,-1.235000,0.600000,-0.484500,-1.307930,-0.750258,-1.121050,-0.392000,-1.734000,0.0,0.500000,0.500000,0.500000,0.000000
max,3.430000,1.930000,3.335000,4.873780,3.878180,2.518550,4.954000,2.170000,0.0,2.000000,2.000000,2.000000,1.000000


In [ ]:
df = pd.read_csv('../expt_phys_data/full_dataset_with_scaffold_split.csv').fillna('')
df = df[(df.smiles != 'C') & (df.smiles != 'S') & (df.smiles != 'N')]
df.describe()

In [11]:
test = df[df.paper_split == 2]
rmsd(test.expt,test.asc)

2.651504488021848

In [7]:
df.scaffold_split.value_counts()

0    479
1     80
2     80
Name: scaffold_split, dtype: int64

In [4]:
split = 'scaffold'
expected_mean = -3.8
scaffolds = df.groupby(['scaffold']).agg({'id':'size', 'expt':'mean','scaffold':'mean'}) 
np.random.seed(37)
a = np.arange(63)
np.random.shuffle(a)
scaffolds['random'] = a

scaffolds.sort_values(by=['id','random'],ascending=False,inplace=True)
scaffolds

TypeError: Could not convert  to numeric

In [149]:
df['scaffold_split'] = 0
no_scaffold = df[df.scaffold==''].id
test_val_samples = no_scaffold.sample(frac=0.25,random_state=37)
test_samples = test_val_samples.sample(frac=0.5,random_state=37)
val_samples=test_val_samples.drop(test_samples.index)
# print(len(test_samples),len(val_samples))
df.loc[df.index[test_samples.index],'scaffold_split'] = 2
df.loc[df.index[val_samples.index],'scaffold_split'] = 1

# df.loc[df.scaffold =='c1ccncc1','scaffold_split'] = 1
# df.loc[df.scaffold =='c1ccc(-c2ccccc2)cc1','scaffold_split'] = 2
# df.loc[df.scaffold =='c1ccc2c(c1)Oc1ccccc1O2','scaffold_split'] = 1
# df.loc[df.scaffold =='C1CCCCC1','scaffold_split'] = 2
# df.loc[df.scaffold =='c1ccncc1'],'scaffold_split'] = 1
# df.loc[df.scaffold =='c1ccncc1'],'scaffold_split'] = 1

In [158]:
split = 'scaffold_split'
expected_mean = -3.8
scaffolds = df.groupby(['scaffold']).agg({'id':'size', 'expt':'mean','scaffold_split':'mean'}) 
np.random.seed(37)
a = np.arange(63)
np.random.shuffle(a)
scaffolds['random'] = a

scaffolds.sort_values(by=['id','random'],ascending=False,inplace=True)
scaffolds

,id,expt,scaffold_split,random
scaffold,,,,
,320,-2.378062,0.375,29
c1ccccc1,152,-4.738750,0.000,53
c1ccncc1,22,-5.359545,2.000,28
c1ccc(-c2ccccc2)cc1,16,-3.858750,1.000,17
c1ccc2ccccc2c1,13,-5.240769,1.000,35
c1ccc2c(c1)Oc1ccccc1O2,12,-3.705000,0.000,21
C1CCCCC1,9,-1.536667,2.000,13
O=c1cc[nH]c(=O)[nH]1,8,-15.463750,0.000,55
O=C1c2ccccc2C(=O)c2ccccc21,5,-11.312000,0.000,39


In [151]:
#modified
train, test, validation = [], [], []
target_ratios = {'train': 0.75, 'test': 0.125, 'validation': 0.125}

# Initialize counters
total_molecules = len(df)
counts = {'train': 0, 'test': 0, 'validation': 0}
total=1
for scaffold, row in scaffolds.iterrows():
    # print(scaffold)
    if scaffold == '':
        continue
    group_size = row['id']
    
    # Choose the partition for the current scaffold based on the current distribution and target ratios
    # current_ratios = {k: v / total for k, v in counts.items()}
    # # Determine which partition is furthest below its target ratio
    # partition = min(target_ratios.keys(), key=lambda x: current_ratios[x] - target_ratios[x])


    current_ratios = {k: (v+group_size) / (total+group_size) for k, v in counts.items()}
    # print(current_ratios)
    # Determine which partition is furthest below its target ratio
    partition = min(target_ratios.keys(), key=lambda x: abs(current_ratios[x] - target_ratios[x]))
    
    # print(partition)
    # Add the current scaffold group to the chosen partition
    if partition == 'train':
        train.append(scaffold)
    elif partition == 'test':
        test.append(scaffold)
    else:
        validation.append(scaffold)
    
    # Update counts
    counts[partition] += group_size
    if(total==1):
        total -=1
    total+=group_size

print(counts)

{'train': 242.0, 'test': 40.0, 'validation': 40.0}


In [152]:
df.loc[(df.scaffold.isin(validation)),'scaffold_split'] =1
df.loc[(df.scaffold.isin(test)),'scaffold_split'] =2

In [108]:
df.describe()

,expt,tip3p,cha,zap9,mbondi,asc,igb5,null,paper_split,hilo_split,scaffold_split
count,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.0,642.000000,642.000000,642.000000
mean,-3.803006,-3.486223,-3.966495,-3.760505,-4.453775,-3.954268,-5.192626,0.0,0.373832,0.373832,0.376947
std,3.847820,4.186301,4.106919,4.122175,4.589629,4.956833,4.750501,0.0,0.695742,0.695742,0.698537
min,-25.470000,-21.762000,-20.490600,-22.504700,-31.910700,-25.772000,-25.222000,0.0,0.000000,0.000000,0.000000
25%,-5.727500,-5.421500,-5.628592,-5.857230,-6.782935,-6.201250,-7.420750,0.0,0.000000,0.000000,0.000000
50%,-3.530000,-3.053500,-3.358170,-3.010565,-3.823930,-3.172000,-4.427000,0.0,0.000000,0.000000,0.000000
75%,-1.215000,-0.480750,-1.298365,-0.749270,-1.120665,-0.390500,-1.732000,0.0,0.000000,0.000000,0.750000
max,3.430000,3.335000,4.873780,3.878180,2.518550,4.954000,2.170000,0.0,2.000000,2.000000,2.000000


In [156]:

phys_models = ['expt','tip3p','cha','zap9','mbondi','asc','igb5','null']
dataset_names = ['full','train','valid','test']

split = 'scaffold_split'
for phys in phys_models:
    print(phys)
    for i, dataset in enumerate(dataset_names):
        if(dataset=='full'):
            temp = df.copy()
        else:
            temp = df[df[split] == i-1]
        # temp = temp[temp.scaffold!='']
        if(phys == 'expt'):
            print(f'{dataset}: len={len(temp)}, mean={temp.expt.mean():.2f}, std={temp.expt.std():.2f}, \
min={temp.expt.min():.2f}, max={temp.expt.max():.2f}')
        else:
            print(f'{dataset}: {rmsd(temp.expt,temp[phys]):.2f}')

expt
full: len=642, mean=-3.80, std=3.85, min=-25.47, max=3.43
train: len=482, mean=-3.94, std=4.05, min=-25.47, max=3.16
valid: len=80, mean=-3.56, std=2.95, min=-10.78, max=3.43
test: len=80, mean=-3.24, std=3.36, min=-10.22, max=3.13
tip3p
full: 1.54
train: 1.51
valid: 1.71
test: 1.54
cha
full: 1.72
train: 1.71
valid: 1.35
test: 2.07
zap9
full: 1.67
train: 1.66
valid: 1.71
test: 1.70
mbondi
full: 2.25
train: 2.20
valid: 2.71
test: 2.09
asc
full: 2.51
train: 2.43
valid: 2.87
test: 2.57
igb5
full: 2.84
train: 2.87
valid: 2.84
test: 2.67
null
full: 5.41
train: 5.64
valid: 4.61
test: 4.65


In [157]:
df.to_csv('../expt_phys_data/full_dataset_with_scaffold_split.csv',index=False)